## GROUNDWATER LEVEL PREDICTOR

### Importing necesaary libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import joblib

### Reading the Dataset

In [2]:
df = pd.read_excel('data1.xlsx')

### Convert columns to numeric, coercing errors

In [3]:
df['LATITUDE'] = pd.to_numeric(df['LATITUDE'], errors='coerce')
df['LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], errors='coerce')
df['WATER LEVEL(mbgl)'] = pd.to_numeric(df['WATER LEVEL(mbgl)'], errors='coerce')

### Drop rows with NaN values

In [4]:
df.dropna(inplace=True)

In [5]:
features = df[['LATITUDE', 'LONGITUDE']]
target = df['WATER LEVEL(mbgl)']

### Normalize features and target

In [6]:
scaler_X = MinMaxScaler()
features_scaled = scaler_X.fit_transform(features)
scaler_y = MinMaxScaler()
target_scaled = scaler_y.fit_transform(target.values.reshape(-1, 1)).flatten()

### Convert to time series format

In [7]:
def create_dataset(X, y, time_step=1):
    Xs, ys = [], []
    for i in range(len(X) - time_step):
        Xs.append(X[i:(i + time_step)])
        ys.append(y[i + time_step])
    return np.array(Xs), np.array(ys)

time_step = 10 
X, y = create_dataset(features_scaled, target_scaled, time_step)

### Split the data into training and testing sets

In [8]:
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

### Model

In [9]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_step, X_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

c:\Users\Surya Prakash\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


### Compiling the model

In [10]:
model.compile(optimizer='adam', loss='mean_squared_error')

### Fitting the Model

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.0011 - val_loss: 0.0129
Epoch 2/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0012 - val_loss: 0.0149
Epoch 3/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0011 - val_loss: 0.0136
Epoch 4/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0013 - val_loss: 0.0134
Epoch 5/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0011 - val_loss: 0.0137
Epoch 6/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0011 - val_loss: 0.0141
Epoch 7/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0011 - val_loss: 0.0144
Epoch 8/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0011 - val_loss: 0.0129
Epoch 9/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0011 - val_loss: 0.0139
Epoch 10/10
315/315 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0011 - val_loss: 0.0144


### Saving the Model

In [12]:
model.save('lstm_model.h5')
joblib.dump(scaler_X, 'scaler_X.pkl')
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']